In [1]:
import random
import numpy
import array

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

import representations, fitness


#INITIAL_BLOCKS = 5 # Represents how many random layer blocks each NNet should start with
INITIAL_BLOCKS = 5 # Random Exclusive represents how many random layer blocks each NNet should start with

POPULATION = 5
GENERATIONS = 2
PROB_MUTATIONS = 0.0 # Probability of mutating in a new generation
PROB_MATE = 0.8 # Probability of mating / crossover in a new generation
NUMBER_EPOCHS = 2 #Epochs when training the network

#---------------------

def getRandomIndividual(iterations=1):
    #Possible networks to choose from:
    '''
    networks = [
        [representations.make_conv2d_repr(),
        representations.make_pool_repr()],

        [representations.make_dropout_repr(),
        representations.make_conv2d_repr()],

        [representations.make_batchnorm_repr()],

        [representations.make_noise_repr()]
    ]

    probabilities = [0.3, 0.3, 0.25, 0.15]

    
    out = []

    
    for x in range(0,iterations):
        choice = numpy.random.choice(networks, p=probabilities)
        for layer in choice:
            out.append(layer)
    
    return out
    '''
    networks = [
        representations.make_conv2d_pool_repr(),
        representations.make_conv2d_dropout_repr(),
        representations.make_batchnorm_repr(),
        representations.make_noise_repr(),
        #representations.make_dropout_repr(),

    ]
    probabilities = [0.3, 0.3, 0.25, 0.15]
    
    return numpy.random.choice(networks,p=probabilities)


'''
Evaluation function (should return the fitness)
'''
def evaluateFunc(individual):
    return fitness.evaluate_nn(individual, NUMBER_EPOCHS), #<--- IMPORTANT: add the comma ','; as it needs to return a tuple

def initRepeatRandom(container, func, n):
    """
    Extended toolbox.initRepeat() function to work with random initialization instead of fixed numbers.
    """
    return container(func() for _ in range(numpy.random.randint(1,n)))

# -------------- Init / Main stuff ----------------------

# Create attributes
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", numpy.ndarray,  fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("individual", initRepeatRandom, creator.Individual, getRandomIndividual, n=INITIAL_BLOCKS) #<-- Creates 3 elements. This random, however does not evaluate the random function each time yet
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


toolbox.register("evaluate", evaluateFunc) #register the evaluation function
toolbox.register("mate", tools.cxTwoPoint)
#toolbox.register("mutate", mutations.mutate_append_remove, prob_remove=1)
toolbox.register("select", tools.selTournament, tournsize=3)
#toolbox.register("select", tools.selBest)
#deap.tools.selBest(individuals, k, fit_attr='fitness')¶



def main():
    random.seed(1337)
    pop = toolbox.population(n=POPULATION)

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit


    hof = tools.HallOfFame(10, similar=numpy.array_equal)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=PROB_MATE, mutpb=PROB_MUTATIONS, ngen=GENERATIONS, stats=stats, halloffame=hof, verbose=True)

    print("\n----------------------------------")
    print(log)
    print("----------------------------------\n")
    print("Best network:")
    print(hof[0])
    print("With a fitness of: ", hof[0].fitness)
    print("\nBad network (%dth):" % 4)
    print(hof[4])
    print("With a fitness of: ", hof[4].fitness)



if __name__ == "__main__":
    main()

c:\users\ruben\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 200 samples, validate on 50 samples
Epoch 1/2
200/200 [==============================] - 1s 4ms/step - loss: 1.7576 - acc: 0.4350 - val_loss: 1.1625 - val_acc: 0.7400
Epoch 2/2
200/200 [==============================] - 0s 1ms/step - loss: 0.6374 - acc: 0.8600 - val_loss: 0.8619 - val_acc: 0.6800
Train on 200 samples, validate on 50 samples
Epoch 1/2
200/200 [==============================] - 0s 1ms/step - loss: 15.1323 - acc: 0.0500 - val_loss: 14.0382 - val_acc: 0.1200
Epoch 2/2
200/200 [==============================] - 0s 115us/step - loss: 14.3153 - acc: 0.1050 - val_loss: 13.3275 - val_acc: 0.1400
Train on 200 samples, validate on 50 samples
Epoch 1/2
200/200 [==============================] - 1s 4ms/step - loss: 1.9919 - acc: 0.3300 - val_loss: 1.3249 - val_acc: 0.6400
Epoch 2/2
200/200 [==============================] - 0s 1ms/step - loss: 0.8937 - acc: 0.8150 - val_loss: 0.8546 - val_acc: 0.8200
Error evaluating..
Train on 200 samples, validate on 50 samples
Epoch 1/2

In [2]:
from representations import INSERTABLE, MUTABLE_PARAMS, default_init_nn_repr
import numpy as np
import random

In [6]:
indiv = toolbox.individual()

In [7]:
indiv[1]

{'params': {'activation': 'relu',
  'filters': 64,
  'kernel_size': 3,
  'rate': 0.34},
 'type': 'conv2ddropout'}

In [8]:
indiv

Individual([{'type': 'conv2ddropout', 'params': {'filters': 16, 'kernel_size': 3, 'activation': 'relu', 'rate': 0.35}},
            {'type': 'conv2ddropout', 'params': {'filters': 64, 'kernel_size': 3, 'activation': 'relu', 'rate': 0.34}},
            {'type': 'noise', 'params': {'stddev': 0.07034362869194766}},
            {'type': 'conv2dpool', 'params': {'filters': 64, 'kernel_size': 3, 'activation': 'relu', 'pool_size': 1}}],
           dtype=object)

In [ ]:
for elem in representations.REPR_MAKERS:
    if indiv[1]['type'] == elem:
        print(representations.REPR_MAKERS[elem])

In [22]:
def mutate_layer(layer, verbose=False):
    '''
    Looks up the initializer function for a type,
    and replaces it with a new initialization.    
    '''
    for elem in representations.REPR_MAKERS:
        if layer['type'] == elem:
            layer = representations.REPR_MAKERS[elem]()
            if verbose:
                print('MUTATED LAYER %s' % layer['type'])
    return layer

In [55]:
def mutate_network(repr, mutations=1, verbose=False):
    '''
    Mutates a whole representation
    '''
    if mutations > len(repr):
        mutations = len(repr)-1 # prevents setting higher count of mutations than length of representation

    if verbose:
        print("MUTATING %d BLOCKS OF NETWORK" % mutations)
    
    for layerIndex in np.random.randint(0, len(repr), mutations):
        if verbose:
            repr[layerIndex] = mutate_layer(repr[layerIndex], verbose=True)
        else:
            repr[layerIndex] = mutate_layer(repr[layerIndex])
    return repr

In [56]:
mutate_layer(indiv[3])

{'params': {'activation': 'relu',
  'filters': 32,
  'kernel_size': 3,
  'pool_size': 4},
 'type': 'conv2dpool'}

In [59]:
print(indiv)
print('----')
print(mutate_network(indiv, 3, verbose=True))

[{'type': 'conv2ddropout', 'params': {'filters': 8, 'kernel_size': 3, 'activation': 'relu', 'rate': 0.39}}
 {'type': 'conv2ddropout', 'params': {'filters': 16, 'kernel_size': 3, 'activation': 'relu', 'rate': 0.24}}
 {'type': 'noise', 'params': {'stddev': 0.2762815910081766}}
 {'type': 'conv2dpool', 'params': {'filters': 8, 'kernel_size': 3, 'activation': 'relu', 'pool_size': 2}}]
----
MUTATING 3 BLOCKS OF NETWORK
MUTATED LAYER noise
MUTATED LAYER conv2ddropout
MUTATED LAYER conv2dpool
[{'type': 'conv2ddropout', 'params': {'filters': 64, 'kernel_size': 3, 'activation': 'relu', 'rate': 0.29}}
 {'type': 'conv2ddropout', 'params': {'filters': 16, 'kernel_size': 3, 'activation': 'relu', 'rate': 0.24}}
 {'type': 'noise', 'params': {'stddev': 0.8524606775501399}}
 {'type': 'conv2dpool', 'params': {'filters': 32, 'kernel_size': 3, 'activation': 'relu', 'pool_size': 2}}]


In [35]:
import numpy as np

In [43]:
mutations = 2
for randindex in np.random.randint(0, len(indiv), mutations):
    print(randindex)
    
# CONTINUE WITH THIS RANDOM SELECTION BLOCK

1
3


In [40]:
len(indiv)

4

In [136]:
import keras
indiv = toolbox.individual()

In [137]:
model = representations.reprs2nn(indiv)

In [138]:
comp = model.count_params()
comp

433290

In [139]:
np.log(comp)

12.979162528763322

In [140]:
0.8 - 0.02 * np.log(comp)

0.5404167494247336